In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [ ]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=1000),    
)
docs = loader.load()

In [ ]:
docs[0].metadata['source']

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=1000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [ ]:
documents[0].page_content

In [ ]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm=ChatOpenAI(model="gpt-4-turbo")


In [ ]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. 
Please follow below instructions to design your new code:
(1) New Code should be based on .NET 8+ or ASP.NET Core. 
(2) Outmost namespace is AspNetCoreApp                                          
(3) Generate the Startup.cs file also
(4) Use Razor Pages page-based programming model to build web UI
                                          
Please follow below instructions to output response:
(1)Please use xml tag <description></description> to contain all your description; and use xml tag <NewCode><file name="xxxx"></file><file name="yyyy"></file>...</NewCode> to wrap source code files generated.
(2)The content within <file name="xxxx"></file> is valid C# code.
                                          
In addition, basedon the context provided, please help to identify.Net Framework features replaced. 

<context>
{context}
</context>
Old Code: {input}"""
)


In [ ]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

In [ ]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [ ]:
response=retriever_chain.invoke({"input": documents[0].page_content})

In [ ]:
response['answer']

In [ ]:
import re
from IPython.display import display, Markdown

# Your input string
input_string = response['answer']



code_match = re.search(r'<NewCode>\n(.*?)\n</NewCode>', input_string, re.DOTALL)
description_match = re.search(r'<description>\n(.*?)\n</description>', input_string, re.DOTALL)

# Extracted code and description
code = code_match.group(1) if code_match else None
description = description_match.group(1) if description_match else None

print(code)


display(Markdown(description))



In [ ]:
pattern = re.compile(r'<file name="(.+?)">(.*?)</file>', re.DOTALL)

# Finding all matches
matches = pattern.findall(code)

# Loop through matches and write to files
for file_name, file_content in matches:
    # Strip leading and trailing whitespaces from the content
    file_content = file_content.strip()
    # Open the file in write mode and write the content
    with open("./DotNet/"+file_name, 'w') as file:
        file.write(file_content)
    print(f"File saved: {file_name}")